# **第3章　学校のクラス編成**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/takazawa/PyOptBookMeijiUnivBA/blob/main/3.school/school_lect.ipynb)

In [ ]:
# Google Colabで実行する場合は初回に必ず実行する
import os
if 'COLAB_GPU' in os.environ:
  if not os.path.exists("PyOptBookMeijiUnivBA"):
    !git clone https://github.com/takazawa/PyOptBookMeijiUnivBA.git
  !cp PyOptBookMeijiUnivBA/*/*.csv .
  !pip install -r PyOptBookMeijiUnivBA/requirements.txt -q

### **3.3 数理モデリングと実装**

### ②データの確認

In [ ]:
# データ処理のためのライブラリpandasの取り込み
import pandas as pd

(1)生徒データ(students.csv)の確認

In [ ]:
# students.csvからの生徒データの取得
s_df = pd.read_csv('students.csv')
print(len(s_df))
s_df.head()

In [ ]:
# 学籍番号の確認
s_df['student_id']

In [ ]:
# 最大値の確認
s_df['student_id'].max()

In [ ]:
# 最小値の確認
s_df['student_id'].min()

In [ ]:
# 1〜318まで隙間なく番号が振られているかの確認
set(range(1, 319)) == set(s_df['student_id'].tolist())

In [ ]:
# 性別（gender）の確認
s_df['gender'].value_counts()

In [ ]:
# 学力試験の点数(score)の統計量の確認
s_df['score'].describe()

In [ ]:
# 学力試験の点数(score)の分布の確認
s_df['score'].hist()

In [ ]:
# リーダー気質フラグ(leader_flag)の確認
s_df['leader_flag'].value_counts()

In [ ]:
# 特別支援フラグ(support_flag)の確認
s_df['support_flag'].value_counts()

(2)特定ペアデータ(student_pairs.csv)の確認

In [ ]:
# student_pairs.csvからの特定ペアデータの取得
s_pair_df = pd.read_csv('student_pairs.csv')
print(len(s_pair_df))
s_pair_df

### ③数理モデリングと実装

In [ ]:
# PythonライブラリPuLPの取り込み
import pulp

In [ ]:
# 数理モデルのインスタンス作成
prob = pulp.LpProblem('ClassAssignmentProblem', pulp.LpMaximize)

In [ ]:
# 生徒のリスト
S = s_df['student_id'].tolist()
print(S)

In [ ]:
# クラスのリスト
C = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
C

In [ ]:
# 生徒とクラスのペアのリスト
SC = [(s,c) for s in S for c in C]
print(SC[:30])

In [ ]:
# 生徒をどのクラスに割り当てるかを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

In [ ]:
# (1)各生徒は１つのクラスに割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1

In [ ]:
# (2)各クラスの生徒の人数は39人以上、40人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) >= 39
    prob += pulp.lpSum([x[s,c] for s in S]) <= 40

In [ ]:
# 男子生徒のリスト
S_male = [row.student_id for row in s_df.itertuples() if row.gender == 1]

# 女子生徒のリスト
S_female = [row.student_id for row in s_df.itertuples() if row.gender == 0]

# (3) 各クラスの男子生徒、女子生徒の人数は20人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_male]) <= 20
    prob += pulp.lpSum([x[s,c] for s in S_female]) <= 20

In [ ]:
# 学力を辞書表現に変換
score = {row.student_id:row.score for row in s_df.itertuples()}

# 平均点の算出
score_mean = s_df['score'].mean()
print(score_mean)
    
# (4) 各クラスの学力試験の平均点は学年平均点±10点とする。    
for c in C:
    prob += (score_mean - 10) * pulp.lpSum([x[s,c] for s in S]) <= pulp.lpSum([x[s,c] * score[s] for s in S]) 
    prob += pulp.lpSum([x[s,c] * score[s] for s in S]) <= (score_mean + 10) * pulp.lpSum([x[s,c] for s in S])

In [ ]:
# リーダー気質の生徒の集合
S_leader = [row.student_id for row in s_df.itertuples() if row.leader_flag == 1]

# (5)各クラスにリーダー気質の生徒を2人以上割り当てる。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_leader]) >= 2

In [ ]:
# 特別な支援が必要な生徒の集合
S_support = [row.student_id for row in s_df.itertuples() if row.support_flag == 1]

# (6) 特別な支援が必要な生徒は各クラスに1人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_support]) <= 1

In [ ]:
# 生徒の特定ペアリスト
SS = [(row.student_id1, row.student_id2) for row in s_pair_df.itertuples()]

# (7) 特定ペアの生徒は同一クラスに割り当てない。
for s1, s2 in SS:
    for c in C:
        prob += x[s1,c] + x[s2,c] <= 1

In [ ]:
# prob # 追記

In [ ]:
# 求解
status = prob.solve()
print(status)
print(pulp.LpStatus[status])

In [ ]:
# 最適化結果の表示
# 各クラスに割り当てられている生徒のリストを辞書に格納
C2Ss = {}
for c in C:
    C2Ss[c] = [s for s in S if x[s,c].value()==1]
            
for c, Ss in C2Ss.items():
    print('Class:', c)
    print('Num:', len(Ss))
    print('Student:', Ss)
    print()

# 演習: 解が要件を満たしているかを確認する

要件(1) 学年の全生徒をそれぞれ①つのクラスに割り当てる

In [ ]:
# 演習

検証用データフレームの作成

In [ ]:
# 検証用のデータフレームの用意
result_df = s_df.copy()

# 各生徒がどのクラスに割り当てられたかの情報を辞書に格納
S2C = {s:c for s in S for c in C if x[s,c].value()==1}

# 生徒データに各生徒がどのクラスに割り当てられたかの情報を結合            
result_df['assigned_class'] = result_df['student_id'].map(S2C)
result_df.head(5)

要件(2) 各クラスの生徒の人数は39人以上、40人以下とする

In [ ]:
# 演習

(3) 各クラスの男子生徒、女子生徒の人数は20人以下とする。

In [ ]:
# 演習

(4) 各クラスの学力試験の平均点は学年平均点±10点とする。

In [ ]:
# 演習

(5) 各クラスにリーダー気質の生徒を2人以上割り当てる。

In [ ]:
# 演習

(6) 特別な支援が必要な生徒は各クラスに1人以下とする。

In [ ]:
# 演習

(7) 特定ペアの生徒は同一クラスに割り当てない。

In [ ]:
# 演習

# 演習: 要件の追加

In [ ]:
# 特定の学生は指定するクラスに割り当てる
fixed_student_class_pairs = [(1, "A"), (3, "B"), (50, "C")]

# 特定の学生は指定するクラスに割り当てない
forbidden_student_class_pair = [(2, "E"), (10, "B")]

# 特定の生徒同士を同じクラスに割り当てる
student_pairs = [(100, 101), (103, 104)]

In [ ]:
import pandas as pd
import pulp

prob = pulp.LpProblem('ClassAssignmentProblem2', pulp.LpMaximize)

# 生徒をどのクラスに割り当てるを変数として定義
x = pulp.LpVariable.dicts('x', SC, cat='Binary')

# (1)各生徒は１つのクラスに割り当てる
for s in S:
    prob += pulp.lpSum([x[s,c] for c in C]) == 1

# (2)各クラスの生徒の人数は39人以上、40人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S]) >= 39
    prob += pulp.lpSum([x[s,c] for s in S]) <= 40

# 男子生徒のリスト
S_male = [row.student_id for row in s_df.itertuples() if row.gender == 1]

# 女子生徒のリスト
S_female = [row.student_id for row in s_df.itertuples() if row.gender == 0]

# (3) 各クラスの男子生徒、女子生徒の人数は20人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_male]) <= 20
    prob += pulp.lpSum([x[s,c] for s in S_female]) <= 20

# 学力を辞書表現に変換
score = {row.student_id:row.score for row in s_df.itertuples()}

# 平均点の算出
score_mean = s_df['score'].mean()

# (4) 各クラスの学力試験の平均点は学年平均点±10点とする。
for c in C:
    prob += pulp.lpSum([x[s,c]*score[s] for s in S]) >= (score_mean - 10) * pulp.lpSum([x[s,c] for s in S])
    prob += pulp.lpSum([x[s,c]*score[s] for s in S]) <= (score_mean + 10) * pulp.lpSum([x[s,c] for s in S])

# リーダー気質の生徒の集合
S_leader = [row.student_id for row in s_df.itertuples() if row.leader_flag == 1]

# (5)各クラスにリーダー気質の生徒を2人以上割り当てる。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_leader]) >= 2

# 特別な支援が必要な生徒の集合
S_support = [row.student_id for row in s_df.itertuples() if row.support_flag == 1]

# (6) 特別な支援が必要な生徒は各クラスに1人以下とする。
for c in C:
    prob += pulp.lpSum([x[s,c] for s in S_support]) <= 1

# 生徒の特定ペアリスト
SS = [(row.student_id1, row.student_id2) for row in s_pair_df.itertuples()]

# (7) 特定ペアの生徒は同一クラスに割り当てない。
for s1, s2 in SS:
    for c in C:
        prob += x[s1,c] + x[s2,c] <= 1

# 追加の制約を記述

# 求解
status = prob.solve()
print('Status:', pulp.LpStatus[status])

# 最適化結果の表示
# 各クラスに割り当てられている生徒のリストを辞書に格納
C2Ss = {}
for c in C:
    C2Ss[c] = [s for s in S if x[s,c].value()==1]

for c, Ss in C2Ss.items():
    print('Class:', c)
    print('Num:', len(Ss))
    print('Student:', Ss)
    print()

In [69]:
# 解の確認